In [1]:
import torch
import torch.nn as nn
import torchtext as T
from torch.hub import load_state_dict_from_url
from torch.utils.data import DataLoader
from datasets import load_dataset, load_metric
import numpy as np
import datasets
import random
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoModelForMaskedLM

DEVICE = torch.device("cuda")

In [2]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "rte"
model_name = "roberta_debiased"
model_class = "roberta"
base = False
if (task == "qnli") | (task == "mnli") | (task == "qqp"):
    batch_size = 8
else:
    batch_size = 16

### Loading and Pre-processing Data

In [3]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

Reusing dataset glue (C:\Users\shahzehan\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
if base == False:
    tokenizer = AutoTokenizer.from_pretrained('tokenizers\{}'.format(model_name),use_fast=True)
else:
    tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [5]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

sentence1_key, sentence2_key = task_to_keys[task]

In [6]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)

In [7]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at C:\Users\shahzehan\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-3223ce83b1cc476c.arrow
Loading cached processed dataset at C:\Users\shahzehan\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-ddc189ae9f32e9d0.arrow
Loading cached processed dataset at C:\Users\shahzehan\.cache\huggingface\datasets\glue\rte\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-37455e2560f1b33b.arrow


### Training and Evaluating Model

In [8]:
num_labels = 3 if task.startswith("mnli") else 1 if task=="stsb" else 2

if base == True:
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
else:
    old_model = AutoModelForMaskedLM.from_pretrained('models\{}'.format(model_name))
    if model_class == "bert":
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)
        model.bert.embeddings = old_model.bert.embeddings
        model.bert.encoder = old_model.bert.encoder
    elif model_class == "roberta":
        model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=num_labels)
        model.roberta.embeddings = old_model.roberta.embeddings
        model.roberta.encoder = old_model.roberta.encoder
    elif model_class == "albert":
        model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2", num_labels=num_labels)
        model.albert.embeddings = old_model.albert.embeddings
        model.albert.encoder = old_model.albert.encoder

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

In [9]:
metric_name = "pearson" if task == "stsb" else "matthews_correlation" if task == "cola" else "accuracy"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [10]:
def compute_metrics(eval_pred):
    global model_name
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [11]:
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation" 
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [12]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\shahzehan\miniconda3\envs\dick2\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2490
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 156


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689854,0.527076


The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 16
Saving model checkpoint to test-glue\checkpoint-156
Configuration saved in test-glue\checkpoint-156\config.json
Model weights saved in test-glue\checkpoint-156\pytorch_model.bin
tokenizer config file saved in test-glue\checkpoint-156\tokenizer_config.json
Special tokens file saved in test-glue\checkpoint-156\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-glue\checkpoint-156 (score: 0.5270758122743683).


TrainOutput(global_step=156, training_loss=0.6968468886155349, metrics={'train_runtime': 35.22, 'train_samples_per_second': 70.699, 'train_steps_per_second': 4.429, 'total_flos': 209505261163080.0, 'train_loss': 0.6968468886155349, 'epoch': 1.0})

In [13]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: idx, sentence2, sentence1. If idx, sentence2, sentence1 are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 16


{'eval_loss': 0.6898541450500488,
 'eval_accuracy': 0.5270758122743683,
 'eval_runtime': 1.1052,
 'eval_samples_per_second': 250.623,
 'eval_steps_per_second': 16.286,
 'epoch': 1.0}

In [14]:
torch.cuda.empty_cache()